In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [132]:
news_copy = news_data.copy()

In [150]:
base_url = "https://www.moneycontrol.com/news/tags/companies.html"
news_data = []

In [149]:
def extractNewsData(news):
    news_json = {}
    if not news.get('class') or 'clearfix' not in news.get('class'):
        return;
    a = news.find('a')
    span = news.find('span')
    
    news_json['link'] = a.get('href')
    news_json['title'] = a.get('title')
    news_json['date time'] = span.text
    
    p = news.find_all('p')[0]
    news_json['desc'] = p.text
    news_json['content'] = getFullNewsText(news_json['link'])
    return news_json

def getFullNewsText(link:str):
    response = requests.get(link)
    soup = BeautifulSoup(response.text,'html.parser')  
    try:
        paras = soup.find_all('div',id='contentdata')[0].find_all('p')
        text = ""
        for para in paras:
            text += para.text
    except Exception as err:
        text = soup.find('p','pro_artidesc').text
    
    return text

def fetchPage(page=None):
    url = base_url
    if page:
        url = f"{base_url}/page-{page}/"
    print(url)
    response = requests.get(url)   
    soup = BeautifulSoup(response.text,'html.parser')   
    news_ul = soup.find_all("ul",id="cagetory")[0]
    if not news_ul:
        return
    li = news_ul.find_all('li')
    for index,news in enumerate(li):
        print(index)
        news_json = extractNewsData(news)
        news_data.append(news_json)
        
def getDateObject(dateString:str,dateFormat='%B %d, %Y %I:%M %p'):
    dt_object = datetime.strptime(dateString, dateFormat)
    return dt_object

In [151]:
fetchPage()

https://www.moneycontrol.com/news/tags/companies.html
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [154]:
for page in range(2,31):
    print("fetching page -- ",page)
    fetchPage(page)

fetching page --  2
https://www.moneycontrol.com/news/tags/companies.html/page-2/
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
fetching page --  3
https://www.moneycontrol.com/news/tags/companies.html/page-3/
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
fetching page --  4
https://www.moneycontrol.com/news/tags/companies.html/page-4/
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
fetching page --  5
https://www.moneycontrol.com/news/tags/companies.html/page-5/
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
fetching page --  6
https://www.moneycontrol.com/news/tags/companies.html/page-6/
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
fetching page --  7
https://www.moneycontrol.com/news/tags/companies.html/page-7/
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1

In [155]:
news_data = [d for d in news_data if d is not None ]
len(news_data)

750

In [158]:
import pandas as pd
df = pd.DataFrame(news_data)
df.to_csv("News.csv",index=False)

In [160]:
getFullNewsText("https://www.moneycontrol.com/news/business/stocks/max-healthcare-should-good-health-come-at-a-price-11884891.html")

'                        Highlights\r\n\r\n\tGood operational and financial performance in Q2FY24\r\n\tARPOB, clinical mix, volumes – all shaping up well\r\n\tExpansion plans to support near-term growth\r\n\tTrading at elevated valuation multiples\r\n\r\nMax Healthcare (Max; CMP: Rs 678; M Cap: Rs 65,909 crore; Rating: Equal-weight) has reported good Q2FY24 earnings as most key operating parameters remained supportive.\r\n\r\nA lot of factors are favourable for the healthcare sector — rising insurance penetration, higher per capita income, preference for quality healthcare, as well as government’s push for better healthcare infrastructure.\r\n\r\nWe reckon...\n                    '

In [164]:
df.iloc[7]['content']

'                        Highlights\r\n\r\n\tMarkets have staged a sharp recovery – Nifty touching Mount 21K\r\n\tOn forward P/E basis, stocks still not in the overvalued zone\r\n\tAverage gap between bond and earnings yield at the peak points to some upside\r\n\tCalendar 2024 to experience volatility mostly emanating from the global front\r\n\tInvestors should add quality stocks at every dip to participate in India’s long-term bull market\r\n\r\nFears of higher rates for longer and a probable US hard landing as well as the strife in the Middle East turning...\n                    '